In [1]:
import duckdb

In [20]:
cn = duckdb.connect("orders.db")

In [ ]:
sql = """
create or replace table order_hdr (order_nbr int, order_crt_dt date, region_office string, ord_qty int, sales_amt decimal(8,2), cogs decimal(8,2));
INSERT INTO order_hdr VALUES
    (1, '2021-01-01', 'North', 7, 23.49, 8.32),
    (2, '2021-02-05', 'North', 4, 86.34, 51.34),
    (3, '2021-02-18', 'West', 12, 67.13, 40.88),
    (4, '2021-03-07', 'West', 10, 103.34, 90.18),
    (5, '2021-05-12', 'South', 6, 12.01, 7.99),
    (6, '2022-05-12', 'North', 23, 18.34, 14.99),
    (7, '2022-05-12', 'South', 1, 5.99, 3.50),
    (8, '2022-05-12', 'West', 3, 27.88, 19.12),
    (9, '2022-05-12', 'East', 7, 46.03, 30.00),
    (10, '2022-05-12', 'North', 3, 137.31, 89.54),
"""
cn.execute(sql)

In [36]:
cn.sql('from order_hdr').show()

┌───────────┬──────────────┬───────────────┬─────────┬──────────────┬──────────────┐
│ order_nbr │ order_crt_dt │ region_office │ ord_qty │  sales_amt   │     cogs     │
│   int32   │     date     │    varchar    │  int32  │ decimal(8,2) │ decimal(8,2) │
├───────────┼──────────────┼───────────────┼─────────┼──────────────┼──────────────┤
│         1 │ 2021-01-01   │ North         │       7 │        23.49 │         8.32 │
│         2 │ 2021-02-05   │ North         │       4 │        86.34 │        51.34 │
│         3 │ 2021-02-18   │ West          │      12 │        67.13 │        40.88 │
│         4 │ 2021-03-07   │ West          │      10 │       103.34 │        90.18 │
│         5 │ 2021-05-12   │ South         │       6 │        12.01 │         7.99 │
│         6 │ 2022-05-12   │ North         │      23 │        18.34 │        14.99 │
│         7 │ 2022-05-12   │ South         │       1 │         5.99 │         3.50 │
│         8 │ 2022-05-12   │ West          │       3 │        27.

### Using qualify statement to grab top order amounts by region

In [37]:
sql = """
select *
from order_hdr
qualify 1 = rank() over(partition by region_office order by sales_amt desc)
"""
cn.sql(sql).show()

┌───────────┬──────────────┬───────────────┬─────────┬──────────────┬──────────────┐
│ order_nbr │ order_crt_dt │ region_office │ ord_qty │  sales_amt   │     cogs     │
│   int32   │     date     │    varchar    │  int32  │ decimal(8,2) │ decimal(8,2) │
├───────────┼──────────────┼───────────────┼─────────┼──────────────┼──────────────┤
│        10 │ 2022-05-12   │ North         │       3 │       137.31 │        89.54 │
│         9 │ 2022-05-12   │ East          │       7 │        46.03 │        30.00 │
│         5 │ 2021-05-12   │ South         │       6 │        12.01 │         7.99 │
│         4 │ 2021-03-07   │ West          │      10 │       103.34 │        90.18 │
└───────────┴──────────────┴───────────────┴─────────┴──────────────┴──────────────┘



### Union All By Name

In [71]:
sql = """
create or replace table order_hdr_legacy (order_nbr int, region_office string, sales_amt decimal(8,2), cogs decimal(8,2));

insert into order_hdr_legacy values 
    (28, 'North', 33.33, 26.41)
    ,(23, 'South', 18.67, 15.59)
    ,(34, 'East', 19.99, 7.29)
;
"""
cn.execute(sql)
cn.sql("from order_hdr_legacy").show()

┌───────────┬───────────────┬──────────────┬──────────────┐
│ order_nbr │ region_office │  sales_amt   │     cogs     │
│   int32   │    varchar    │ decimal(8,2) │ decimal(8,2) │
├───────────┼───────────────┼──────────────┼──────────────┤
│        28 │ North         │        33.33 │        26.41 │
│        23 │ South         │        18.67 │        15.59 │
│        34 │ East          │        19.99 │         7.29 │
└───────────┴───────────────┴──────────────┴──────────────┘



In [72]:
sql = """
create or replace table order_hdr_combined
as
select *
from order_hdr
union all by name
select *
from order_hdr_legacy
"""

cn.execute(sql)
cn.sql("from order_hdr_combined").show()


┌───────────┬──────────────┬───────────────┬─────────┬──────────────┬──────────────┐
│ order_nbr │ order_crt_dt │ region_office │ ord_qty │  sales_amt   │     cogs     │
│   int32   │     date     │    varchar    │  int32  │ decimal(8,2) │ decimal(8,2) │
├───────────┼──────────────┼───────────────┼─────────┼──────────────┼──────────────┤
│         1 │ 2021-01-01   │ North         │       7 │        23.49 │         8.32 │
│         2 │ 2021-02-05   │ North         │       4 │        86.34 │        51.34 │
│         3 │ 2021-02-18   │ West          │      12 │        67.13 │        40.88 │
│         4 │ 2021-03-07   │ West          │      10 │       103.34 │        90.18 │
│         5 │ 2021-05-12   │ South         │       6 │        12.01 │         7.99 │
│         6 │ 2022-05-12   │ North         │      23 │        18.34 │        14.99 │
│         7 │ 2022-05-12   │ South         │       1 │         5.99 │         3.50 │
│         8 │ 2022-05-12   │ West          │       3 │        27.

In [44]:
cn.sql("from order_hdr_combined").show()

┌───────────┬──────────────┬───────────────┬─────────┬──────────────┬──────────────┐
│ order_nbr │ order_crt_dt │ region_office │ ord_qty │  sales_amt   │     cogs     │
│   int32   │     date     │    varchar    │  int32  │ decimal(8,2) │ decimal(8,2) │
├───────────┼──────────────┼───────────────┼─────────┼──────────────┼──────────────┤
│         1 │ 2021-01-01   │ North         │       7 │        23.49 │         8.32 │
│         2 │ 2021-02-05   │ North         │       4 │        86.34 │        51.34 │
│         3 │ 2021-02-18   │ West          │      12 │        67.13 │        40.88 │
│         4 │ 2021-03-07   │ West          │      10 │       103.34 │        90.18 │
│         5 │ 2021-05-12   │ South         │       6 │        12.01 │         7.99 │
│         6 │ 2022-05-12   │ North         │      23 │        18.34 │        14.99 │
│         7 │ 2022-05-12   │ South         │       1 │         5.99 │         3.50 │
│         8 │ 2022-05-12   │ West          │       3 │        27.

### Lateral Alias

In [73]:
sql = """
select region_office
    ,tot_sales - tot_cogs as gross_profit
    ,concat(round((tot_sales - tot_cogs)/tot_sales*100,2),'%') as gross_margin
from (
    select region_office, sum(sales_amt) as tot_sales, sum(cogs) as tot_cogs
    from order_hdr_combined
    group by all
) as sub
order by region_office
"""
cn.sql(sql).show()

┌───────────────┬───────────────┬──────────────┐
│ region_office │ gross_profit  │ gross_margin │
│    varchar    │ decimal(38,2) │   varchar    │
├───────────────┼───────────────┼──────────────┤
│ East          │         28.73 │ 43.52%       │
│ North         │        108.21 │ 36.21%       │
│ South         │          9.59 │ 26.15%       │
│ West          │         48.17 │ 24.29%       │
└───────────────┴───────────────┴──────────────┘



In [48]:
sql = """
select region_office
    ,tot_sales - tot_cogs as gross_profit
    ,concat(round(gross_profit/tot_sales*100,2),'%') as gross_margin
from (
    select region_office, sum(sales_amt) as tot_sales, sum(cogs) as tot_cogs
    from order_hdr_combined
    group by all
) as sub
order by region_office
"""
cn.sql(sql).show()

┌───────────────┬───────────────┬──────────────┐
│ region_office │ gross_profit  │ gross_margin │
│    varchar    │ decimal(38,2) │   varchar    │
├───────────────┼───────────────┼──────────────┤
│ East          │         28.73 │ 43.52%       │
│ North         │        108.21 │ 36.21%       │
│ South         │          9.59 │ 26.15%       │
│ West          │         48.17 │ 24.29%       │
└───────────────┴───────────────┴──────────────┘



### Dynamic Pivot

In [49]:
sql = """
with data as (select extract(year from order_crt_dt) as year, region_office, sales_amt from order_hdr_combined)

pivot data
on region_office
using sum(sales_amt)
"""
cn.sql(sql).show()

┌───────┬───────────────┬───────────────┬───────────────┬───────────────┐
│ year  │     East      │     North     │     South     │     West      │
│ int64 │ decimal(38,2) │ decimal(38,2) │ decimal(38,2) │ decimal(38,2) │
├───────┼───────────────┼───────────────┼───────────────┼───────────────┤
│  NULL │         19.99 │         33.33 │         18.67 │          NULL │
│  2021 │          NULL │        109.83 │         12.01 │        170.47 │
│  2022 │         46.03 │        155.65 │          5.99 │         27.88 │
└───────┴───────────────┴───────────────┴───────────────┴───────────────┘



In [76]:
sql = """
with data as (select extract(year from order_crt_dt) as year, region_office, sales_amt from order_hdr_combined)

select year, "East", "North", "South", "West"
from data
pivot (sum(sales_amt) for region_office in ("North","South","East","West"))
"""
cn.sql(sql).show()

┌───────┬───────────────┬───────────────┬───────────────┬───────────────┐
│ year  │     East      │     North     │     South     │     West      │
│ int64 │ decimal(38,2) │ decimal(38,2) │ decimal(38,2) │ decimal(38,2) │
├───────┼───────────────┼───────────────┼───────────────┼───────────────┤
│  NULL │         19.99 │         33.33 │         18.67 │          NULL │
│  2021 │          NULL │        109.83 │         12.01 │        170.47 │
│  2022 │         46.03 │        155.65 │          5.99 │         27.88 │
└───────┴───────────────┴───────────────┴───────────────┴───────────────┘



In [ ]:

declare @sql nvarchar(max);
declare @list nvarchar(max);

set @list = STUFF((select concat('[', some_col,']' for xml path ('')),1,1);

set @sql = N'select grouping_col, @list from some_table pivot(sum(some_other col) for col_list in @list)

exec sp_executesql @sql


### Excluding columns from final output

In [70]:
sql = """
    select * exclude("75_percentile", "95_percentile")
    from (
        select
            *
            ,case when "75_percentile" >= 25 then 'good' else 'bad' end as "75_ind"
            ,case when "75_percentile" >= 30 then 'good' else 'bad' end as "95_ind"
        from (
            select
                region_office
                ,sum(sales_amt - cogs) as gross_profit
                ,round(gross_profit/sum(sales_amt)*100,2) as gross_margin
                ,gross_margin * .95 as "95_percentile"
                ,gross_margin * .75 as "75_percentile"
            from order_hdr_combined
            group by all
        ) as sub
    ) as sub2
    order by 1
"""
cn.sql(sql).show()

┌───────────────┬───────────────┬──────────────┬─────────┬─────────┐
│ region_office │ gross_profit  │ gross_margin │ 75_ind  │ 95_ind  │
│    varchar    │ decimal(38,2) │    double    │ varchar │ varchar │
├───────────────┼───────────────┼──────────────┼─────────┼─────────┤
│ East          │         28.73 │        43.52 │ good    │ good    │
│ North         │        108.21 │        36.21 │ good    │ bad     │
│ South         │          9.59 │        26.15 │ bad     │ bad     │
│ West          │         48.17 │        24.29 │ bad     │ bad     │
└───────────────┴───────────────┴──────────────┴─────────┴─────────┘



In [ ]:
sql = """
    SELECT
        ORDER_NBR, ORDER_CRT_DT, REGION_OFFICE, STORE_NBR, ORD_AMT, ORD_QTY
    FROM order_hdr_modern
    UNION ALL
    SELECT
        ORDER_NBR, ORDER_CRT_DT, CAST(NULL AS VARCHAR(10)) AS REGION_OFFICE, CAST(NULL AS INT) AS STORE_NBR, ORD_AMT, CAST(NULL AS INT) AS ORD_QTY
    FROM order_hdr_legacy


"""